### Add 100 pedestrians

In [2]:
import matplotlib.pyplot as plt	
import matplotlib.image as mpimg	
import numpy as np	
import glob	
import os	
import sys	
import time	
try:	
    sys.path.append(glob.glob('PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (	
        sys.version_info.major,	
        sys.version_info.minor,	
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])	
except IndexError:	
    pass	
import carla	
import random	
import cv2	
import skimage.measure as measure	
#in synchronous mode, sensor data must be added to a queue	
import queue	
client = carla.Client('localhost', 2000)	
client.set_timeout(11.0)	
world = client.load_world('Town03')	
settings = world.get_settings()	
settings.fixed_delta_seconds = 0.05 #must be less than 0.1, or else physics will be noisy	
#must use fixed delta seconds and synchronous mode for python api controlled sim, or else 	
#camera and sensor data may not match simulation properly and will be noisy 	
settings.synchronous_mode = True 	
weather = carla.WeatherParameters(	
    cloudiness=0.0,	
    precipitation=0.0,	
    sun_altitude_angle=90.0)	
world.set_weather(weather)	
world.apply_settings(settings)

#add pedestrians on the road
blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')
spawn_points = world.get_map().get_spawn_points()
walkers_list = []
for i in range(100):
    try:
        spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
        walker_bp = random.choice(blueprints)
        walker = world.spawn_actor(walker_bp, spawn_point)
        walkers_list.append(walker)
    except Exception:
        pass
    # walker.set_autopilot()

#add a BMW car and set it on autopilot
blueprints = world.get_blueprint_library().filter('vehicle.bmw.grandtourer')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.spawn_actor(blueprints[0], spawn_points[0])
vehicle.set_autopilot()

transform = random.choice(world.get_map().get_spawn_points()) 
m= world.get_map()
waypoint = m.get_waypoint(transform.location)

#attach a sensor rgb camera to the bmw
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-15))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

#save images generated by this camera
im_width = 1920
im_height = 1080
image_queue = queue.Queue()
camera.listen(image_queue.put)

dataset_dicts = []
global_count=0
timestamp = round(time.time())
print(timestamp)
for i in range(1000):
    #step
    world.tick()

    #rgb camera
    image = image_queue.get()

    
    if i%100==0:
        image.save_to_disk(f"test_images/{timestamp}/%06d.png" %(image.frame))

        # img = mpimg.imread(f"test_images/{timestamp}/%06d.png" %(image.frame))

    #drive vehicle to next waypoint on map
    waypoint = random.choice(waypoint.next(1.5))
    vehicle.set_transform(waypoint.transform)



1669916702


### Add 200 vehicles and set them to autopilot

In [1]:
import matplotlib.pyplot as plt	
import matplotlib.image as mpimg	
import numpy as np	
import glob	
import os	
import sys	
import time	
try:	
    sys.path.append(glob.glob('PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (	
        sys.version_info.major,	
        sys.version_info.minor,	
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])	
except IndexError:	
    pass	
import carla	
import random	
import cv2	
import skimage.measure as measure	
#in synchronous mode, sensor data must be added to a queue	
import queue	
client = carla.Client('localhost', 2000)	
client.set_timeout(11.0)	
world = client.load_world('Town03')	
settings = world.get_settings()	
settings.fixed_delta_seconds = 0.05 #must be less than 0.1, or else physics will be noisy	
#must use fixed delta seconds and synchronous mode for python api controlled sim, or else 	
#camera and sensor data may not match simulation properly and will be noisy 	
settings.synchronous_mode = True 	
weather = carla.WeatherParameters(	
    cloudiness=0.0,	
    precipitation=0.0,	
    sun_altitude_angle=90.0)	
world.set_weather(weather)	
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.audi.a2')) # lets choose a vehicle at random

# lets choose a random spawn point
transform = random.choice(world.get_map().get_spawn_points()) 

#spawn a vehicle
vehicle = world.spawn_actor(bp, transform) 

actor_list=[]
#add 20 vehicles and set them to autopilot
vehicles_list = []
count_v=0
for i in range(200):
    blueprints = random.choice(world.get_blueprint_library().filter('vehicle'))
    spawn_points = random.choice(world.get_map().get_spawn_points())
    other_vehicle = world.try_spawn_actor(blueprints, spawn_points)
    if other_vehicle is not None:
    # vehicle = world.spawn_actor(blueprints[0], spawn_points[0])
        other_vehicle.set_autopilot()
        actor_list.append(other_vehicle)
        count_v+=1

print(count_v)

#add pedestrians on the road
blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')
spawn_points = world.get_map().get_spawn_points()
walkers_list = []
for i in range(5):
    try:
        spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
        walker_bp = random.choice(blueprints)
        walker = world.spawn_actor(walker_bp, spawn_point)
        actor_list.append(walker)
    except Exception:
        pass
    # walker.set_autopilot()


transform = random.choice(world.get_map().get_spawn_points()) 
m= world.get_map()
waypoint = m.get_waypoint(transform.location)

#attach a sensor rgb camera to the bmw
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-15))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

#save images generated by this camera
im_width = 1920
im_height = 1080
image_queue = queue.Queue()
camera.listen(image_queue.put)

dataset_dicts = []
global_count=0
timestamp = round(time.time())
for i in range(1000):
    #step
    world.tick()

    #rgb camera
    image = image_queue.get()

    
    if i%100==0:
        image.save_to_disk(f"test_images/{timestamp}/%06d.png" %(image.frame))

        # img = mpimg.imread(f"test_images/{timestamp}/%06d.png" %(image.frame))

    #drive vehicle to next waypoint on map
    waypoint = random.choice(waypoint.next(1.5))
    vehicle.set_transform(waypoint.transform)


138


### Add trees

In [1]:
import matplotlib.pyplot as plt	
import matplotlib.image as mpimg	
import numpy as np	
import glob	
import os	
import sys	
import time	
try:	
    sys.path.append(glob.glob('PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (	
        sys.version_info.major,	
        sys.version_info.minor,	
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])	
except IndexError:	
    pass	
import carla	
import random	
import cv2	
import skimage.measure as measure	
#in synchronous mode, sensor data must be added to a queue	
import queue	
client = carla.Client('localhost', 2000)	
client.set_timeout(11.0)	
world = client.load_world('Town03')	
settings = world.get_settings()	
settings.fixed_delta_seconds = 0.05 #must be less than 0.1, or else physics will be noisy	
#must use fixed delta seconds and synchronous mode for python api controlled sim, or else 	
#camera and sensor data may not match simulation properly and will be noisy 	
settings.synchronous_mode = True 	
weather = carla.WeatherParameters(	
    cloudiness=0.0,	
    precipitation=0.0,	
    sun_altitude_angle=90.0)	
world.set_weather(weather)	
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.audi.a2')) # lets choose a vehicle at random

# lets choose a random spawn point
transform = random.choice(world.get_map().get_spawn_points()) 

#spawn a vehicle
vehicle = world.spawn_actor(bp, transform) 

actor_list=[]
#add 20 vehicles and set them to autopilot
vehicles_list = []
count_v=0
for i in range(100):
    blueprints = random.choice(world.get_blueprint_library().filter('vehicle'))
    spawn_points = random.choice(world.get_map().get_spawn_points())
    other_vehicle = world.try_spawn_actor(blueprints, spawn_points)
    if other_vehicle is not None:
    # vehicle = world.spawn_actor(blueprints[0], spawn_points[0])
        other_vehicle.set_autopilot()
        actor_list.append(other_vehicle)
        count_v+=1

print(count_v)

#add pedestrians on the road
blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')
spawn_points = world.get_map().get_spawn_points()
walkers_list = []
for i in range(10):
    try:
        spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
        walker_bp = random.choice(blueprints)
        walker = world.spawn_actor(walker_bp, spawn_point)
        actor_list.append(walker)
    except Exception:
        pass

#add trees
blueprints = world.get_blueprint_library().filter('static.prop.tree')
spawn_points = world.get_map().get_spawn_points()
trees_list = []
for i in range(10):
    try:
        spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
        tree_bp = random.choice(blueprints)
        tree = world.spawn_actor(tree_bp, spawn_point)
        actor_list.append(tree)
    except Exception:
        pass

    # walker.set_autopilot()


transform = random.choice(world.get_map().get_spawn_points()) 
m= world.get_map()
waypoint = m.get_waypoint(transform.location)

#attach a sensor rgb camera to the bmw
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '90')
camera_transform = carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-15))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

#save images generated by this camera
im_width = 1920
im_height = 1080
image_queue = queue.Queue()
camera.listen(image_queue.put)

dataset_dicts = []
global_count=0
timestamp = round(time.time())
for i in range(100):
    #step
    world.tick()

    #rgb camera
    image = image_queue.get()

    
    if i%10==0:
        image.save_to_disk(f"test_images/{timestamp}/%06d.png" %(image.frame))

        # img = mpimg.imread(f"test_images/{timestamp}/%06d.png" %(image.frame))

    #drive vehicle to next waypoint on map
    waypoint = random.choice(waypoint.next(1.5))
    vehicle.set_transform(waypoint.transform)


90
